# Final Project

## Introduction
blabla  




## Algorithm Implementation
In this part we will implement the Huffman and LZ77 algorithms required for the task
### Huffman


In [36]:
from collections import Counter
import heapq

class Huffman(object):
    
    def __init__(self):
        pass
    
    def generate_code(self, char_stream):
        counts= Counter(char_stream)
        print counts
        heap = []
        
        #construct an heap
        for char, freq in counts.items():
            node = {'val': char, 'left': None, 'right': None}
            heapq.heappush(heap, (freq, node))
            
        #build a prefix tree
        while len(heap) > 1:
            fr1, left = heapq.heappop(heap)
            fr2, right = heapq.heappop(heap)
            node = { 'left': left, 'right': right}
            heapq.heappush(heap, (fr1 + fr2, node))
        print heap
        _, root = heapq.heappop(heap)
        code_dict = {}
        print root
        self.traverse(root, '', code_dict)
        print code_dict
    
    def traverse(self, node, prefix, code_dict):
        if 'val' in node:
            code_dict[node['val']] = prefix
        else:
            self.traverse(node['left'], (prefix + '0'), code_dict)
            self.traverse(node['right'], prefix + '1', code_dict)
        
        

huff = Huffman()
l =  ['t']*27 + ['H'] * 7 + ['j'] * 5 + ['h'] * 4 + ['o'] * 57

huff.generate_code(l)
            


Counter({'o': 57, 't': 27, 'H': 7, 'j': 5, 'h': 4})
[(100, {'right': {'right': None, 'val': 'o', 'left': None}, 'left': {'right': {'right': None, 'val': 't', 'left': None}, 'left': {'right': {'right': {'right': None, 'val': 'j', 'left': None}, 'left': {'right': None, 'val': 'h', 'left': None}}, 'left': {'right': None, 'val': 'H', 'left': None}}}})]
{'right': {'right': None, 'val': 'o', 'left': None}, 'left': {'right': {'right': None, 'val': 't', 'left': None}, 'left': {'right': {'right': {'right': None, 'val': 'j', 'left': None}, 'left': {'right': None, 'val': 'h', 'left': None}}, 'left': {'right': None, 'val': 'H', 'left': None}}}}
{'H': '000', 'j': '0011', 't': '01', 'o': '1', 'h': '0010'}


## Data sets  
### Data set 1  - Run Or Walk
#### A dataset containing labeled sensor data from accelerometer and gyroscope 
https://www.kaggle.com/vmalyi/run-or-walk  
#### Data exploration  
The data stored in csv format, sized 7589889 bytes and contains 88589 samples.  
It includes header line with column description:  
date, time, username, wrist id, activity id (label) metadata and acceleration and gyro sensor data for 3d axes  

Lets load the data and preview few lines:

In [2]:
import numpy as np
import pandas as pd
df=pd.read_csv('../datasets/runOrWalk/dataset.csv')
df.head()

,date,time,username,wrist,activity,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z
0,2017-6-30,13:51:15:847724020,viktor,0,0,0.2650,-0.7814,-0.0076,-0.0590,0.0325,-2.9296
1,2017-6-30,13:51:16:246945023,viktor,0,0,0.6722,-1.1233,-0.2344,-0.1757,0.0208,0.1269
2,2017-6-30,13:51:16:446233987,viktor,0,0,0.4399,-1.4817,0.0722,-0.9105,0.1063,-2.4367
3,2017-6-30,13:51:16:646117985,viktor,0,0,0.3031,-0.8125,0.0888,0.1199,-0.4099,-2.9336
4,2017-6-30,13:51:16:846738994,viktor,0,0,0.4814,-0.9312,0.0359,0.0527,0.4379,2.4922


Let's see data types detected by pandas:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88588 entries, 0 to 88587
Data columns (total 11 columns):
date              88588 non-null object
time              88588 non-null object
username          88588 non-null object
wrist             88588 non-null int64
activity          88588 non-null int64
acceleration_x    88588 non-null float64
acceleration_y    88588 non-null float64
acceleration_z    88588 non-null float64
gyro_x            88588 non-null float64
gyro_y            88588 non-null float64
gyro_z            88588 non-null float64
dtypes: float64(6), int64(2), object(3)
memory usage: 7.4+ MB


### Data set 2  
blabla  
### Data set 3  
blabla  
